In [1]:
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
import geopandas as gp
import numpy as np

In [15]:
surplus = pd.read_csv('data/surplus.csv')
distance_matrix = pd.DataFrame([],columns=['county_A','county_B','distance'])
#surplus = surplus[:10]

In [3]:
def distance_geog(lat1,lon1,lat2,lon2):
    # http://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude-python
    # approximate radius of earth in miles
    R = 3959.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [4]:
import numpy as np
vfunc = np.vectorize(distance_geog)
x1, y1, x2, y2 = np.array(surplus.lon), np.array(surplus.lat), np.array(surplus.lon), np.array(surplus.lat)

In [6]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.

    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.

    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.

    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])

    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [7]:
vectors = np.hstack([cartesian([x1,x2]),cartesian([y1,y2])])

In [8]:
#x_1,x_2,y_1,y_2 = vectors
x_1,x_2,y_1,y_2 = vectors[:,0],vectors[:,1],vectors[:,2],vectors[:,3]

In [19]:
distances = vfunc(y_1,x_1,y_2,x_2)

In [10]:
counties = cartesian([np.array(surplus['County Name']),np.array(surplus['County Name'])])

In [16]:
distance_matrix['county_A'] = counties[:,0]
distance_matrix['county_B'] = counties[:,1]

In [20]:
distance_matrix['distance'] = distances

In [23]:
fipses = cartesian([np.array(surplus['FIPS']),np.array(surplus['FIPS'])])

In [24]:
distance_matrix['FIPS_A'] = fipses[:,0]
distance_matrix['FIPS_B'] = fipses[:,1]

In [25]:
distance_matrix.to_csv('data/distance_matrix.csv')

In [26]:
distance_matrix.head()

,county_A,county_B,distance,FIPS_A,FIPS_B
0,Autauga,Autauga,0.000000,1001,1001
1,Autauga,Baldwin,140.370274,1001,1003
2,Autauga,Barbour,86.545903,1001,1005
3,Autauga,Bibb,42.446217,1001,1007
4,Autauga,Blount,100.213979,1001,1009
